In [1]:
from pprint import pprint
from tqdm import tqdm
import io
import numpy as np
import pandas as pd
import requests

In [2]:
def hubeau_get_stations_ades(numéro_département: int) -> pd.DataFrame:
    """Récupérer la liste (fichier CSV) des points d'eau d'un département et la retourner sous forme d'un DataFrame.
    Exemple de requête :
    https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/stations.csv?code_departement=66&size=200
    """
    url = 'https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/stations.csv'
    grand_nombre_magique = 9999
    query_params = {'code_departement': numéro_département, 'size': grand_nombre_magique}
    response = requests.get(url, params=query_params, )
    df_liste_stations = pd.read_csv(io.BytesIO(response.content), sep=';')
    return df_liste_stations

In [3]:
def get_chroniques_station(infos_station: dict) -> pd.DataFrame:
    """Récupérer les données d'intérêt sur le code BSS d'une station.
    Exemple de requête :
    https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques.csv?code_bss=10908X0136/DUCUP2&size=20000
    """
    url = 'https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques.csv'
    # grand_nombre_magique = 20000
    query_params = {
        'code_bss': infos_station.get('code_bss'), 'size': infos_station.get('nb_mesures_piezo')}
    response = requests.get(url, params=query_params)
    if response.headers.get('Content-Length') == 0 \
            or 'Content-Encoding' not in response.headers:
        return None
    df = pd.read_csv(io.BytesIO(response.content), sep=';')
    return df

In [4]:
def hubeau_get_chroniques_des_stations(df_liste_stations: pd.DataFrame, nom_fichier_destination: str) -> pd.DataFrame:
    """Récupérer, pour chaque station de la liste, les données d'intérêt et les sauvegarder itérativement dans un fichier CSV
    """
    infos_stations_liste = df_liste_stations.to_dict(orient='records')
    garder_lentête = True
    with open(nom_fichier_destination, 'w', newline='\n') as fichier_csv_cible:
        fichier_csv_cible.write('')
    with tqdm(total=len(infos_stations_liste), ncols=100) as progr_bar:
        for infos_station in infos_stations_liste[:]:
            progr_bar.set_description(f"Station {infos_station.get('code_bss')}")
            df_chroniques_station = get_chroniques_station(infos_station)
            progr_bar.update()
            if df_chroniques_station is None:
                continue
            df_tmp = pd.merge(left=pd.DataFrame([infos_station]),
                              right=df_chroniques_station,
                              on=['code_bss', 'urn_bss'], how='outer')
            df_tmp.to_csv(nom_fichier_destination, mode='a',
                          sep=';', index=False, header=garder_lentête)
            garder_lentête = False
    return pd.read_csv(nom_fichier_destination, sep=';', low_memory=False)

In [2]:
numéro_département = 66
nom_fichier_destination = f'hubeau_chroniques_des_stations_ades_dans_{numéro_département}.csv'

In [6]:
df_liste_stations = hubeau_get_stations_ades(numéro_département)
print(f"Il y a {len(df_liste_stations)} stations dans le département {numéro_département}.")
df_liste_stations_avec_mesures = df_liste_stations[df_liste_stations['nb_mesures_piezo'] > 0]
print(f"Il y a {len(df_liste_stations_avec_mesures)} stations pour lesquelles il existe au moins une mesure enregistrée.")
df_infos_stations_et_chroniques = hubeau_get_chroniques_des_stations(df_liste_stations_avec_mesures, nom_fichier_destination)

Il y a 86 stations dans le département 66.
Il y a 83 stations pour lesquelles il existe au moins une mesure enregistrée.


Station 10915X0395/PZ: 100%|████████████████████████████████████████| 83/83 [02:54<00:00,  2.10s/it]


In [7]:
# df_infos_stations_et_chroniques = pd.read_csv(nom_fichier_destination, sep=';', low_memory=False)
colonnes_dans_lordre = ['code_bss', 'date_debut_mesure', 'date_fin_mesure',
                        'code_commune_insee', 'nom_commune', 'x', 'y', 'codes_bdlisa',
                        'bss_id', 'altitude_station', 'nb_mesures_piezo',
                        'code_departement', 'nom_departement', 'libelle_pe',
                        'profondeur_investigation', 'codes_masse_eau_edl', 'noms_masse_eau_edl',
                        'date_maj', 'date_mesure', 'timestamp_mesure',
                        'niveau_nappe_eau', 'mode_obtention', 'statut', 'qualification',
                        'code_continuite', 'nom_continuite', 'code_producteur',
                        'nom_producteur', 'code_nature_mesure', 'nom_nature_mesure',
                        'profondeur_nappe', 'urn_bss', 'urns_bdlisa', 'urns_masse_eau_edl']
df_infos_stations_et_chroniques = df_infos_stations_et_chroniques[colonnes_dans_lordre]
colonnes_renommées = {'x': 'longitude_x', 'y': 'latitude_y'}
df_infos_stations_et_chroniques.rename(columns=colonnes_renommées, inplace=True)
niveau_min = df_infos_stations_et_chroniques['niveau_nappe_eau'].min()
niveau_max = df_infos_stations_et_chroniques['niveau_nappe_eau'].max()
calcul_IPS = lambda x: (x['niveau_nappe_eau'] - niveau_min) / (niveau_max - niveau_min)
df_infos_stations_et_chroniques['indicateur_piezometrique_standardise'] = df_infos_stations_et_chroniques.apply(calcul_IPS, axis=1)

In [8]:
df_infos_stations_et_chroniques.to_csv(nom_fichier_destination, sep=';', index=False)

Valider le fichier CSV produit en comptant le nombre de champs de chaque ligne:

In [15]:
field_sep = ';'
header = ''
how_many_fields_in_header = 0
with open(nom_fichier_destination, 'r') as fichier_csv_cible:
    for i, line in enumerate(fichier_csv_cible, start=1):
        if i == 1:
            header = line
            how_many_fields_in_header = line.count(field_sep)
        if line.count(field_sep) != how_many_fields_in_header:
            print(str(i).zfill(10), ':', header)
            print(str(i).zfill(10), ':', line)
            break